In [ ]:
import json
file_path = "/home/xhl/eval/my_eval/data/math/train.jsonl"

problems = []
answers = []

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        problems.append(item["problem"])
        answers.append(item["answer"])
examples = ["Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + prompt + "\nAssistant: <think>" for prompt in problems]

In [ ]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from repeng import ControlVector, ControlModel, DatasetEntry
import time

model_name = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(model_name)
print(model)
model = model.to(
    "cuda:1"
    if torch.cuda.is_available()
    else "mps:0"
    if torch.backends.mps.is_available()
    else "cpu"
)
model = ControlModel(model, list(range(1,28)))
settings = {
    "pad_token_id": tokenizer.eos_token_id,  # silence warning
    "do_sample": False,  # temperature=0
    "max_new_tokens": 2048,
    "early_stopping": True,
    # "repetition_penalty": 1.1,  # reduce control jank
}
loaded_vector = ControlVector.import_gguf("execution_avg_vector.gguf")
model.set_control(loaded_vector, 0)

In [ ]:
inputs = []
total_output_tokens=0
for i in range(10):
    input_ids = tokenizer(examples[i], return_tensors="pt").to(model.device)
    inputs.append(input_ids)
time1 = time.time()
for i in range(10):
    output = model.generate(**inputs[i], **settings)
    total_output_tokens += len(output.squeeze())-len(inputs[i][0])
time2 = time.time()
print(total_output_tokens)
print(f"{(time2 - time1)/10:.8f}s/req")
toks_per_sec = total_output_tokens / (time2 - time1) if time2 - time1 > 0 else 0
print(f"output toks/s: {toks_per_sec:.2f}")
# print(tokenizer.decode(output.squeeze()))